In this code we load the excel data set and first divide them into train validate and test to later train an already pretrained model with tuning it to give us the results we need.
first we load the data from google drive.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from transformers import pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
import torch


# ✅ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ✅ Define file path
script_dir = "/content/drive/MyDrive/data"   # make sure 'data' is the correct folder name
file_name = "voice_command_dataset_10000.csv"
file_path = os.path.join(script_dir, file_name)

# ✅ Read CSV using full path
df = pd.read_csv(file_path)

# ✅ Display info
print(df.head())
print(df['intent'].value_counts())


X = df['command']       # input text
y = df['intent']        # target labels

# First split into train + temp (temp = val + test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Then split temp into validation and test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# ✅ Print sizes
print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")
print(f"Test samples: {len(X_test)}")

Mounted at /content/drive
   id intent                             command
0   1  stand                please stand quickly
1   2  stand          come on, stand tall slowly
2   3  stand  come to a standing position for me
3   4  stand           wake up and stand alright
4   5  stand             come on, stand tall now
intent
stand          1250
sit            1250
go_forward     1250
go_left        1250
go_right       1250
go_back        1250
turn_around    1250
do_a_trick     1250
Name: count, dtype: int64
Training samples: 7000
Validation samples: 1500
Test samples: 1500


Below is an example of how a pre trained model perform given a text and our intents where it gives a score(probability) for what it most likely is.

In [ ]:


classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
sequence = "please move forward a few steps"
candidate_labels = ["stand", "sit", "go_forward", "go_left", "go_right", "go_back", "turn_around", "do_a_trick"]
result = classifier(sequence, candidate_labels)
print(result)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


{'sequence': 'please move forward a few steps', 'labels': ['go_forward', 'stand', 'go_right', 'turn_around', 'do_a_trick', 'go_left', 'go_back', 'sit'], 'scores': [0.5919970273971558, 0.12286117672920227, 0.08198466897010803, 0.07599616795778275, 0.05695445090532303, 0.03589024394750595, 0.021014539524912834, 0.013301664032042027]}


in this code we try to edit the pre trained model and train it on out datset but it gives some errors so we try another time

In [ ]:
# ✅ Prepare Hugging Face Datasets
train_dataset = Dataset.from_dict({"text": X_train.tolist(), "label": y_train.tolist()})
val_dataset = Dataset.from_dict({"text": X_val.tolist(), "label": y_val.tolist()})

# ✅ Encode string labels → numeric IDs
label_encoder = LabelEncoder()
label_encoder.fit(df['intent'])
num_labels = len(label_encoder.classes_)

def encode_labels(example):
    return {"label": int(label_encoder.transform([example["label"]])[0])}

train_dataset = train_dataset.map(encode_labels)
val_dataset = val_dataset.map(encode_labels)

# ✅ Tokenization
model_name = "facebook/bart-large-mnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# ✅ Load model (handles mismatch between pretrained NLI head and your 8 intents)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    ignore_mismatched_sizes=True
)

# ✅ Training configuration (Colab-friendly)
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

# ✅ Metrics (fixed tuple issue)
def compute_metrics(pred):
    labels = pred.label_ids
    logits = pred.predictions
    if isinstance(logits, tuple):  # unwrap tuple if needed
        logits = logits[0]
    preds = logits.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# ✅ Train
trainer.train()

# ✅ Evaluate
eval_results = trainer.evaluate()
print("\n✨ Validation Results:", eval_results)

# ✅ Optional: test a new command
test_text = "please move forward a few steps"
inputs = tokenizer(test_text, return_tensors="pt", truncation=True, padding=True)
outputs = model(**inputs)
logits = outputs.logits if hasattr(outputs, "logits") else outputs[0]
pred_label = label_encoder.inverse_transform([logits.argmax(-1).item()])[0]
print("Predicted intent:", pred_label)



Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-large-mnli and are newly initialized because the shapes did not match:
- classification_head.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([8]) in the model instantiated
- classification_head.out_proj.weight: found shape torch.Size([3, 1024]) in the checkpoint and torch.Size([8, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-3635534268.py:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.088600,0.000039,1.000000,1.000000
2,0.000100,0.000020,1.000000,1.000000
3,0.000000,0.000016,1.000000,1.000000


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight'].



✨ Validation Results: {'eval_loss': 3.874924368574284e-05, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_runtime': 52.0492, 'eval_samples_per_second': 28.819, 'eval_steps_per_second': 3.612, 'epoch': 3.0}


RuntimeError: Expected all tensors to be on the same device, but got index is on cpu, different from other tensors on cuda:0 (when checking argument in method wrapper_CUDA__index_select)

This time it finishes cleanly where we save the model to test later in the code manually(we can see that it gave very high accuracy score which might mean it overfitted the data)

In [ ]:
# ----------------------------
# Prepare Hugging Face Datasets
# ----------------------------
train_dataset = Dataset.from_dict({"text": X_train.tolist(), "label": y_train.tolist()})
val_dataset = Dataset.from_dict({"text": X_val.tolist(), "label": y_val.tolist()})

# ----------------------------
# Encode string labels → numeric IDs
# ----------------------------
label_encoder = LabelEncoder()
label_encoder.fit(df['intent'])
num_labels = len(label_encoder.classes_)

def encode_labels(example):
    return {"label": int(label_encoder.transform([example["label"]])[0])}

train_dataset = train_dataset.map(encode_labels)
val_dataset = val_dataset.map(encode_labels)

# ----------------------------
# Tokenization
# ----------------------------
model_name = "facebook/bart-large-mnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# ----------------------------
# Load model (fix mismatch between pretrained NLI head and 8 intents)
# ----------------------------
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    ignore_mismatched_sizes=True
)

# ----------------------------
# Training configuration
# ----------------------------
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

# ----------------------------
# Metrics (fixed tuple issue)
# ----------------------------
def compute_metrics(pred):
    labels = pred.label_ids
    logits = pred.predictions
    if isinstance(logits, tuple):
        logits = logits[0]
    preds = logits.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}

# ----------------------------
# Trainer
# ----------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# ----------------------------
# Train
# ----------------------------
trainer.train()

# ----------------------------
# Evaluate
# ----------------------------
eval_results = trainer.evaluate()
print("\n✨ Validation Results:", eval_results)

# ----------------------------
# Predict a new command (handles GPU automatically)
# ----------------------------
test_text = "please move forward a few steps"
inputs = tokenizer(test_text, return_tensors="pt", truncation=True, padding=True)

# Move inputs to same device as model
device = next(model.parameters()).device
inputs = {k: v.to(device) for k, v in inputs.items()}

# Forward pass
outputs = model(**inputs)
logits = outputs.logits if hasattr(outputs, "logits") else outputs[0]

# Predicted label
pred_label = label_encoder.inverse_transform([logits.argmax(-1).item()])[0]
print("Predicted intent:", pred_label)


Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-large-mnli and are newly initialized because the shapes did not match:
- classification_head.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([8]) in the model instantiated
- classification_head.out_proj.weight: found shape torch.Size([3, 1024]) in the checkpoint and torch.Size([8, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-139303927.py:78: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.097600,0.000044,1.000000,1.000000
2,0.000100,0.000023,1.000000,1.000000
3,0.000000,0.000018,1.000000,1.000000


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight'].



✨ Validation Results: {'eval_loss': 4.3858002754859626e-05, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_runtime': 51.7751, 'eval_samples_per_second': 28.971, 'eval_steps_per_second': 3.631, 'epoch': 3.0}
Predicted intent: go_forward


We make an interactive module where we give the command in text and it gives the probability of what it is for each intent, it worked great for the commands that where given as can be seen.

In [ ]:
import torch
import torch.nn.functional as F

def predict_intent(text, model, tokenizer, label_encoder):
    """
    Predict intent and probabilities for a given text.
    """
    # Tokenize
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Move to device
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Forward pass
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits if hasattr(outputs, "logits") else outputs[0]

    # Convert logits → probabilities
    probs = F.softmax(logits, dim=-1).cpu().numpy()[0]

    # Map probabilities to labels
    intent_scores = {label_encoder.inverse_transform([i])[0]: float(prob)
                     for i, prob in enumerate(probs)}

    # Predicted intent
    pred_intent = max(intent_scores, key=intent_scores.get)

    return pred_intent, intent_scores

# ----------------------------
# Example usage
# ----------------------------
while True:
    text = input("Enter a command (or 'quit' to exit): ")
    if text.lower() in ["quit", "exit"]:
        break
    pred_intent, scores = predict_intent(text, model, tokenizer, label_encoder)
    print(f"Predicted intent: {pred_intent}")
    print("Scores:", scores)
    print("-"*50)


Enter a command (or 'quit' to exit): sleep
Predicted intent: sit
Scores: {'do_a_trick': 0.0008177431882359087, 'go_back': 0.0011102083371952176, 'go_forward': 0.005318368785083294, 'go_left': 0.00022846701904200017, 'go_right': 0.001033493084833026, 'sit': 0.9902633428573608, 'stand': 0.00015279157378245145, 'turn_around': 0.0010754435788840055}
--------------------------------------------------
Enter a command (or 'quit' to exit): perform 
Predicted intent: do_a_trick
Scores: {'do_a_trick': 0.993233859539032, 'go_back': 4.325726331444457e-05, 'go_forward': 0.004468567669391632, 'go_left': 0.00026052509201690555, 'go_right': 0.0002787392004393041, 'sit': 0.00023901496024336666, 'stand': 0.0011699582682922482, 'turn_around': 0.0003059772134292871}
--------------------------------------------------
Enter a command (or 'quit' to exit): advance
Predicted intent: go_forward
Scores: {'do_a_trick': 1.6674439393682405e-05, 'go_back': 2.191958083130885e-05, 'go_forward': 0.9998931884765625, 'go